In [ ]:
#Scraping data
import pandas as pd
import numpy as np
from google_play_scraper import Sort, reviews

result, continuation_token = reviews(
    'com.telkom.indihome.external',
    lang='id',
    country='id',
    sort=Sort.MOST_RELEVANT,
    count=5000,
    filter_score_with=None
)

In [ ]:
#membuat dataframe dari fungsi np
df_busu = pd.DataFrame(np.array(result),columns=['review'])

#menggabungkan dataframe sebelumnya dengan cara ngelist data
df_busu = df_busu.join(pd.DataFrame(df_busu.pop('review').tolist()))

df_busu.head()

In [ ]:
len(df_busu.index)

In [ ]:
df_busu[['userName', 'score','at', 'content']].head()

In [ ]:
new_df = df_busu[['userName', 'score','at', 'content']]

# menentukan urutan penyortiran. Jika False, DataFrame akan disortir dalam urutan menurun (descending). Jika True, urutannya adalah menaik (ascending)
sorted_df = new_df.sort_values(by='at', ascending=False)
sorted_df.head()

In [ ]:
my_df = sorted_df[['userName', 'score','at', 'content']]
my_df= my_df[['content', 'score']]
my_df.head()

In [ ]:
my_df.to_csv("C:\dedek\data_myindihome\myindihome5.csv", index = False)

### **Labelling**

In [ ]:
my_df = pd.read_csv('C:\dedek\data_myindihome\myindihome.csv')
my_df.head(10)

In [ ]:
def pelabelan(score):
  if score < 3:
    return 'Negatif'
  elif score == 4 :
    return 'Positif'
  elif score == 5 :
    return 'Positif'
my_df['Label'] = my_df ['score'].apply(pelabelan)
my_df.head(10)

In [ ]:
len(my_df)

In [ ]:
my_df.to_csv("C:\dedek\data_myindihome\myindihome_pelabelan.csv", index = False)

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
my_df = pd.read_csv('C:\dedek\data_myindihome\myindihome_pelabelan.csv')
my_df.head(10)

In [ ]:
my_df.to_csv("C:\dedek\data_myindihome\myindihomepreprocessing.csv", index = False)

### **Normalisasi**

In [ ]:
import pandas as pd
my_df = pd.read_csv('C:\dedek\data_myindihome\myindihomepreprocessing.csv')
my_df.head(10)

In [ ]:
import re
import string

def clean_text(my_df, text_field, new_text_field_name):

    # Ubah semua teks menjadi huruf kecil
    my_df[new_text_field_name] = my_df[text_field].str.lower()
    # Menambahkan spasi sebelum setiap huruf besar dan ubah menjadi huruf kecil
    my_df[new_text_field_name] = my_df[new_text_field_name].apply(lambda elem: re.sub(r'(?<!^)(?=[A-Z])', ' ', elem).lower())
    # Menambahkan spasi sebelum tanda baca
    my_df[new_text_field_name] = my_df[new_text_field_name].apply(lambda elem: re.sub(r'([^\w\s])', r' \1 ', elem))
    # Hapus username, simbol, URL, dan kata-kata yang diawali dengan "rt" atau "http"
    my_df[new_text_field_name] = my_df[new_text_field_name].apply(lambda elem: re.sub(r'(@[A-Za-z0-9_]+)|(\w+:\/\/\S+)|\b(rt|http.+?)\b', '', elem))
    # Hapus angka
    my_df[new_text_field_name] = my_df[new_text_field_name].apply(lambda elem: re.sub(r'\d+', '', elem))
    # Hapus simbol yang tidak diinginkan (tanda baca) yang sudah diberi spasi sebelumnya
    my_df[new_text_field_name] = my_df[new_text_field_name].apply(lambda elem: re.sub(r'[^0-9a-zA-Z\s]', '', elem))
    return my_df

my_df['text_clean'] = my_df['content'].str.lower()
data_clean = clean_text(my_df, 'content', 'text_clean')
data_clean.head(10)

### **Tokenizing**

In [ ]:
import re
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

def tokenize(text_clean):
    return re.findall(r'\b\w+\b', text_clean)#mencocokkan seluruh kata dalam string menggunakan regex

data_clean['token'] = data_clean['text_clean'].apply(tokenize, word_tokenize)
data_clean.head(10)

### **Filtering**

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

# Membuat instance dari StopWordRemoverFactory
factory = StopWordRemoverFactory()
# Membuat stop word remover
stopword_remover = factory.create_stop_word_remover()

# Mendapatkan stopwords bahasa Indonesia
stop_words = set(stopwords.words('indonesian'))

indihome_stopwords = {
'pelayannya', 'ok', 'sudh', 'klo', 'via', 'ui', 'tpi', 'mbps','rusa', 'dg', 'ram', 'bangsaatttt', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z' 'yuk', 'yukk', 'ditelp', 'yuotube', 'yutub', 'ywdh', 'zaman', 'zeh', 'zonk', 'zoom', 'aamiin',  'aapek',  'abai',  'abal',  'abang',  'abar', 'abcd',  'abis',  'abisss',  'abonemen', 'aduh', 'hadeuh', 'jak', 'dipang', 'ngebug', 'bug', 'delay', 'aduh', 'pret', 'dan', 'atau', 'tetapi', 'karena', 'jika', 'sementara', 'sehingga', 'yang', 'daripada', 'sejak', 'setelah', 'sebelum', 'hingga', 'yg', 'seperti', 'contoh', 'di', 'untuk', 'gak', 'nya', 'gk', 'jk', 'assalamu', 'alaikum', 'warahmatullahi', 'wr', 'wb', 'salam', 'pagi', 'siang', 'ga', 'malam', 'adalah', 'adapun', 'agaknya', 'akan', 'akhir', 'akhirnya', 'akulah', 'amatlah', 'andalah', 'antara', 'apa', 'apabila', 'apalagi', 'artinya', 'asalkan', 'atau', 'ataupun', 'awalnya', 'bagaikan', 'bagaimanakah', 'bagi', 'bahkan', 'bahwasanya', 'bakal', 'balik', 'bapak', 'bawah', 'begini', 'beginikah', 'begitu', 'begitulah', 'bekerja', 'belakangan', 'belumlah', 'benarkah', 'berada', 'berakhirlah', 'berapa', 'berapalah', 'berarti', 'berbagai', 'beri', 'berikut', 'berjumlah', 'berkata', 'berkeinginan', 'berlainan', 'berlangsung', 'bermacam', 'bermaksud', 'bersama', 'bersiap', 'bertanya', 'berturut', 'bertutur', 'berupa', 'betul', 'biasa', 'bila', 'bisa', 'boleh', 'bolehlah', 'bukan', 'bukanlah', 'bulan', 'cara', 'cukup', 'cukuplah', 'dahulu', 'dan', 'dari', 'datang', 'demi', 'demikianlah', 'depan', 'dia', 'diakhirinya', 'diantara', 'diberi', 'diberikannya', 'dibuatnya', 'didatangkan', 'diibaratkan', 'diingat', 'diinginkan', 'dijelaskan', 'dikarenakan', 'dikatakannya', 'diketahui', 'dikira', 'dilalui', 'dimaksud', 'dimaksudkannya', 'diminta', 'dimisalkan', 'dimulailah', 'dimungkinkan', 'dipastikan', 'diperbuatnya', 'diperkirakan', 'diperlukan', 'dipersoalkan', 'dipunyai', 'dirinya', 'disebut', 'disebutkannya', 'disinilah', 'ditandaskan', 'ditanyai', 'ditegaskan', 'ditunjuk', 'ditunjukkan', 'ditunjuknya', 'dituturkannya', 'diucapkannya', 'dong', 'dulu', 'enggak', 'entah', 'guna', 'hal', 'hanya', 'hari', 'haruslah', 'hendak', 'hendaknya', 'ia', 'ibarat', 'ibaratnya', 'ikut', 'ingat-ingat', 'inginkah', 'ini', 'inilah', 'itukah', 'jadi', 'jadinya', 'jangankan', 'jauh', 'jawaban', 'jelas', 'jelaslah', 'jika', 'juga', 'jumlahnya', 'kala', 'kalaulah', 'kalian', 'kamilah', 'kamulah', 'kapan', 'kapanpun', 'karenanya', 'kata', 'katakanlah', 'ke', 'kebetulan', 'kedua', 'keinginan', 'kelihatan', 'kelima', 'kembali', 'kemungkinan', 'kenapa', 'kepadanya', 'keseluruhan', 'keterlaluan', 'khususnya', 'kinilah', 'kira-kira', 'kita', 'kok', 'lagi', 'lah', 'lainnya', 'lama', 'lanjut', 'lebih', 'lima', 'macam', 'makanya', 'malah', 'mampu', 'mana', 'manalagi', 'masalah', 'masih', 'masing', 'mau', 'melainkan', 'melalui', 'melihatnya', 'memastikan', 'memberikan', 'memerlukan', 'meminta', 'memisalkan', 'mempergunakan', 'memperlihatkan', 'mempersoalkan', 'mempunyai', 'memungkinkan', 'menambahkan', 'menanti', 'menantikan', 'menanyai', 'mendapat', 'mendatang', 'mendatangkan', 'mengakhiri', 'mengatakan', 'mengenai', 'mengetahui', 'menghendaki', 'mengibaratkannya', 'mengingatkan', 'mengira', 'mengucapkannya', 'menjadi', 'menjelaskan', 'menunjuk', 'menunjukkan', 'menurut', 'menyampaikan', 'menyatakan', 'menyeluruh', 'merasa', 'merekalah', 'meski', 'meyakini', 'minta', 'misal', 'misalnya', 'mulai', 'mulanya', 'mungkinkah', 'naik', 'nanti', 'nyaris', 'oleh', 'pada', 'padanya', 'paling', 'pantas', 'pasti', 'penting', 'per', 'perlu', 'perlunya', 'persoalan', 'pertama-tama', 'pertanyakan', 'pihaknya', 'pula', 'punya', 'rasanya', 'rupanya', 'saatnya', 'sajalah', 'sama', 'sambil', 'sampai-sampai', 'sana', 'sangatlah', 'saya', 'se', 'sebabnya', 'sebagaimana', 'sebagian', 'sebaik-baiknya', 'sebaliknya', 'sebegini', 'sebelum', 'sebenarnya', 'sebesar', 'sebisanya', 'sebut', 'sebutnya', 'secukupnya', 'sedangkan', 'sedikit', 'seenaknya', 'segalanya', 'seharusnya', 'seingat', 'sejauh', 'sejumlah', 'sekadarnya', 'sekali-kali', 'sekaligus', 'sekarang', 'sekecil', 'sekiranya', 'sekitarnya', 'sekurangnya', 'selain', 'selalu', 'selama-lamanya', 'selanjutnya', 'seluruhnya', 'semakin', 'semampunya', 'semasih', 'semata-mata', 'sementara', 'semisalnya', 'semua', 'semula', 'sendirian', 'seolah', 'seorang', 'sepantasnya', 'seperlunya', 'sepertinya', 'sering', 'serta', 'sesaat', 'sesampai', 'sesekali', 'sesuatu', 'sesudah', 'setelah', 'setengah', 'setiap', 'setibanya', 'setidaknya', 'seusai', 'siap', 'siapakah', 'sini', 'soal', 'suatu', 'sudahkah', 'supaya', 'tadinya', 'tahun', 'tambah', 'tampak', 'tandas', 'tanpa', 'tanyakan', 'tapi', 'tegasnya', 'tempat', 'tentang', 'tentulah', 'tepat', 'terasa', 'terdahulu', 'terdiri', 'terhadapnya', 'teringat-ingat', 'terjadilah', 'terkira', 'terlebih', 'termasuk', 'tersampaikan', 'tersebutlah', 'tertuju', 'terutama', 'tetapi', 'tiba', 'tidak', 'tidaklah', 'tinggi', 'tunjuk', 'tutur', 'ucap', 'ujar', 'umum', 'ungkap', 'untuk', 'usai', 'wah', 'waktu', 'walau', 'wong', 'yakin', 'yang', 'ada', 'adanya', 'anda', 'apakah', 'atas', 'akankah', 'antar', 'ataukah', 'bagaimana', 'baik', 'baru', 'belum', 'bagai', 'beginilah', 'bilakah', 'buat', 'bukankah', 'contohnya', 'cuma', 'dapat', 'daripada', 'di', 'dilakukan', 'dialah', 'dini', 'harus', 'hanyalah', 'harusnya', 'hendaklah', 'itu', 'itulah', 'ingin', 'inginkan', 'jangan', 'kalau', 'kali', 'kamu', 'karena', 'katanya', 'kemudian', 'ketika', 'kalaupun', 'kapankah', 'kiranya', 'langsung', 'luar', 'lamanya', 'mengapa', 'mereka', 'makin', 'manakala', 'masihkah', 'mungkinlah', 'nah', 'nantinya', 'paparnya', 'pernah', 'pun', 'pastilah', 'saja', 'saling', 'sampai', 'sebab', 'secara', 'sejak', 'sendiri', 'seperti', 'seseorang', 'sebagainya', 'sebanyak', 'sebegitu', 'sebetulnya', 'sedemikian', 'sedikitnya', 'sekali', 'seketika', 'selagi', 'semaunya', 'semuanya', 'sendirinya', 'sepanjang', 'sepantasnyalah', 'serupa', 'sesama', 'sesuatunya', 'sesudahnya', 'seterusnya', 'sewaktu', 'tandasnya', 'terhadap', 'tersebut', 'tetap', 'tadi', 'tentu', 'tentunya', 'tertentu', 'tidakkah', 'yaitu', 'waduh', 'wahai', 'wakil', 'waktunya', 'warga', 'wib', 'rt', 'no', 'nyatanya', 'olehnya', 'padahal', 'pak', 'panjang', 'para', 'pentingnya', 'percuma', 'perlukah', 'pertama', 'pertanyaan', 'pihak', 'pukul', 'rasa', 'rata', 'saat', 'sama-sama', 'sampaikan', 'sangat', 'satu', 'sayalah', 'sebagai', 'sebaik', 'sebaiknya', 'sebelumnya', 'seberapa', 'sebuah', 'sebutlah', 'sedang', 'segala', 'segera', 'sehingga', 'sejenak', 'sekadar', 'sekalian', 'sekalipun', 'sekitar', 'sekurang-kurangnya', 'sela', 'selaku', 'selama', 'selamanya', 'seluruh', 'semacam', 'semampu', 'semasa', 'semata', 'semisal', 'sempat', 'seolah-olah', 'sepihak', 'seringnya', 'sesegera', 'setempat', 'setiba', 'setidak-tidaknya', 'setinggi', 'siapa', 'siapapun', 'sinilah', 'soalnya', 'sudah', 'sudahlah', 'tahu', 'tak', 'tambahnya', 'tampaknya', 'tanya', 'tanyanya', 'tegas', 'telah', 'tengah', 'terakhir', 'terbanyak', 'terdapat', 'teringat', 'terjadi', 'terjadinya', 'terlalu', 'terlihat', 'ternyata', 'terus', 'tiap', 'tiba-tiba', 'tiga', 'toh', 'turut', 'tuturnya', 'ucapnya', 'ujarnya', 'umumnya', 'ungkapnya', 'usah', 'waktunya', 'walaupun', 'yakni', 'dengan', 'orang', 'bahwa', 'namun', 'dua', 'kepada', 'lalu', 'lain', 'banyak', 'beberapa', 'besar', 'merupakan', 'agar', 'persen', 'wib', 'diri', 'minggu', 'the', 'selasa', 'jumlah', 'kondisi', 'hubungan', 'acara', 'masa', 'hidup', 'senin', 'maupun', 'mantan', 'jenis', 'juni', 'tinggal', 'asal', 'sesuai', 'berat', 'memberi', 'sabtu', 'mencari', 'ruang', 'biasanya', 'berdasarkan', 'pekan', 'membawa', 'tingkat', 'dekat', 'ketiga', 'ribu', 'membantu', 'khusus', 'ditemukan', 'kegiatan', 'tampil', 'bertemu', 'justru', 'menyebutkan', 'milik', 'menjalani', 'sumber', 'upaya', 'mengambil', 'lewat', 'meningkatkan', 'kehidupan', 'penggunaan', 'menghadapi', 'aku', 'kami', 'beliau', 'lo', 'lu', 'kemana', 'gimana', 'kurang', 'kemarin', 'yg', 'ya', 'emang', 'gitu', 'amat', 'sekian', 'sekedar', '&', 'telanjur', 'dimana', 'kah', 'kecuali', 'seraya', 'dsb', 'dll', 'dulunya', 'demikian', 'nggak', 'oh', 'tolong', 'bagaimanapun', 'ada', 'adalah', 'adanya', 'adapun', 'agak', 'agaknya', 'agar', 'akan', 'akankah', 'akhir', 'akhiri', 'akhirnya', 'aku', 'akulah', 'amat', 'amatlah', 'anda', 'andalah', 'antar', 'antara', 'antaranya', 'apa', 'apaan', 'apabila', 'ni', 'apakah', 'apalagi', 'apatah', 'artinya', 'asal', 'asalkan', 'atas', 'atau', 'ataukah', 'ataupun', 'awal', 'awalnya', 'bagai', 'bagaikan', 'bagaimana', 'bagaimanakah', 'bagaimanapun', 'bagi', 'bagian', 'bahkan', 'bahwa', 'bahwasanya', 'baik', 'bakal', 'bakalan', 'balik', 'banyak', 'bapak', 'baru', 'bawah', 'beberapa', 'begini', 'beginian', 'beginikah', 'beginilah', 'begitu', 'begitukah', 'begitulah', 'begitupun', 'bekerja', 'belakang', 'belakangan', 'belum', 'belumlah', 'benar', 'benarkah', 'benarlah', 'berada', 'berakhir', 'berakhirlah', 'berakhirnya', 'berapa', 'berapakah', 'berapalah', 'berapapun', 'berarti', 'berawal', 'berbagai', 'berdatangan', 'beri', 'berikan', 'berikut', 'berikutnya', 'berjumlah', 'berkali-kali', 'berkata', 'berkehendak', 'berkeinginan', 'berkenaan', 'berlainan', 'berlalu', 'berlangsung', 'berlebihan', 'bermacam', 'bermacam-macam', 'bermaksud', 'bermula', 'bersama', 'bersama-sama', 'bersiap', 'bersiap-siap', 'bertanya', 'bertanya-tanya', 'berturut', 'berturut-turut', 'bertutur', 'berujar', 'berupa', 'besar', 'betul', 'betulkah', 'biasa', 'biasanya', 'bila', 'bilakah', 'bisa', 'bisakah', 'boleh', 'bolehkah', 'bolehlah', 'buat', 'bukan', 'bukankah', 'bukanlah', 'bukannya', 'bulan', 'bung', 'cara', 'caranya', 'cukup', 'cukupkah', 'cukuplah', 'cuma', 'dahulu', 'dalam', 'dan', 'dapat', 'dari', 'daripada', 'datang', 'dekat', 'demi', 'demikian', 'demikianlah', 'dengan', 'depan', 'di', 'dia', 'diakhiri', 'diakhirinya', 'dialah', 'diantara', 'diantaranya', 'diberi', 'diberikan', 'diberikannya', 'dibuat', 'dibuatnya', 'didapat', 'didatangkan', 'digunakan', 'diibaratkan', 'diibaratkannya', 'diingat', 'diingatkan', 'diinginkan', 'dijawab', 'dijelaskan', 'dijelaskannya', 'dikarenakan', 'dikatakan', 'dikatakannya', 'dikerjakan', 'diketahui', 'diketahuinya', 'dikira', 'dilakukan', 'dilalui', 'dilihat', 'dimaksud', 'dimaksudkan', 'dimaksudkannya', 'dimaksudnya', 'diminta', 'dimintai', 'dimisalkan', 'dimulai', 'dimulailah', 'dimulainya', 'dimungkinkan', 'dini', 'dipastikan', 'diperbuat', 'diperbuatnya', 'dipergunakan', 'diperkirakan', 'diperlihatkan', 'diperlukan', 'diperlukannya', 'dipersoalkan', 'dipertanyakan', 'dipunyai', 'diri', 'dirinya', 'disampaikan', 'disebut', 'disebutkan', 'disebutkannya', 'disini', 'disinilah', 'ditambahkan', 'ditandaskan', 'ditanya', 'ditanyai', 'ditanyakan', 'ditegaskan', 'ditujukan', 'ditunjuk', 'ditunjuki', 'ditunjukkan', 'ditunjukkannya', 'ditunjuknya', 'dituturkan', 'dituturkannya', 'diucapkan', 'diucapkannya', 'diungkapkan', 'dong', 'dua', 'dulu', 'empat', 'enggak', 'enggaknya', 'entah', 'entahlah', 'guna', 'gunakan', 'hal', 'hampir', 'hanya', 'hanyalah', 'hari', 'harus', 'haruslah', 'harusnya', 'hendak', 'hendaklah', 'hendaknya', 'hingga', 'ia', 'ialah', 'ibarat', 'ibaratkan', 'ibaratnya', 'ibu', 'ikut', 'ingat', 'ingat-ingat', 'ingin', 'inginkah', 'inginkan', 'ini', 'inikah', 'inilah', 'itu', 'itukah', 'itulah', 'jadi', 'jadilah', 'jadinya', 'jangan', 'jangankan', 'janganlah', 'jauh', 'jawab', 'jawaban', 'jawabnya', 'jelas', 'jelaskan', 'jelaslah', 'jelasnya', 'jika', 'jikalau', 'juga', 'jumlah', 'jumlahnya', 'justru', 'kala', 'kalau', 'kalaulah', 'kalaupun', 'kalian', 'kami', 'kamilah', 'kamu', 'kamulah', 'kan', 'kapan', 'kapankah', 'kapanpun', 'karena', 'karenanya', 'kasus', 'kata', 'katakan', 'katakanlah', 'katanya', 'ke', 'keadaan', 'kebetulan', 'kecil', 'kedua', 'keduanya', 'keinginan', 'kelamaan', 'kelihatan', 'kelihatannya', 'kelima', 'keluar', 'kembali', 'kemudian', 'kemungkinan', 'kemungkinannya', 'kenapa', 'kepada', 'kepadanya', 'kesampaian', 'keseluruhan', 'keseluruhannya', 'keterlaluan', 'ketika', 'khususnya', 'kini', 'kinilah', 'kira', 'kira-kira', 'kiranya', 'kita', 'kitalah', 'kok', 'kurang', 'lagi', 'lagian', 'lah', 'lain', 'lainnya', 'lalu', 'lama', 'lamanya', 'lanjut', 'lanjutnya', 'lebih', 'lewat', 'lima', 'luar', 'macam', 'maka', 'makanya', 'makin', 'malah', 'malahan', 'mampu', 'mampukah', 'mana', 'manakala', 'manalagi', 'masa', 'masalah', 'masalahnya', 'masih', 'masihkah', 'masing', 'masing-masing', 'mau', 'maupun', 'melainkan', 'melakukan', 'melalui', 'melihat', 'melihatnya', 'memang', 'memastikan', 'memberi', 'memberikan', 'membuat', 'memerlukan', 'memihak', 'meminta', 'memintakan', 'memisalkan', 'memperbuat', 'mempergunakan', 'memperkirakan', 'memperlihatkan', 'mempersiapkan', 'mempersoalkan', 'mempertanyakan', 'mempunyai', 'memulai', 'memungkinkan', 'menaiki', 'menambahkan', 'menandaskan', 'menanti', 'menanti-nanti', 'menantikan', 'menanya', 'menanyai', 'menanyakan', 'mendapat', 'mendapatkan', 'mendatang', 'mendatangi', 'mendatangkan', 'menegaskan', 'mengakhiri', 'mengapa', 'mengatakan', 'mengatakannya', 'mengenai', 'mengerjakan', 'mengetahui', 'menggunakan', 'menghendaki', 'mengibaratkan', 'mengibaratkannya', 'mengingat', 'mengingatkan', 'menginginkan', 'mengira', 'mengucapkan', 'mengucapkannya', 'mengungkapkan', 'menjadi', 'menjawab', 'menjelaskan', 'menuju', 'menunjuk', 'menunjuki', 'menunjukkan', 'menunjuknya', 'menurut', 'menuturkan', 'menyampaikan', 'menyangkut', 'menyatakan', 'menyebutkan', 'menyeluruh', 'menyiapkan', 'merasa', 'mereka', 'merekalah', 'merupakan', 'meski', 'meskipun', 'meyakini', 'meyakinkan', 'minta', 'mirip', 'misal', 'misalkan', 'misalnya', 'mula', 'mulai', 'mulailah', 'mulanya', 'mungkin', 'mungkinkah', 'nah', 'naik', 'namun', 'nanti', 'nantinya', 'nyaris', 'nyatanya', 'oleh', 'olehnya', 'pada', 'padahal', 'padanya', 'pak', 'paling', 'panjang', 'pantas', 'para', 'pasti', 'pastilah', 'penting', 'pentingnya', 'per', 'percuma', 'perlu', 'perlukah', 'perlunya', 'pernah', 'persoalan', 'pertama', 'pertama-tama', 'pertanyaan', 'pertanyakan', 'pihak', 'pihaknya', 'pukul', 'pula', 'pun', 'punya', 'rasa', 'rasanya', 'rata', 'rupanya', 'saat', 'saatnya', 'saja', 'sajalah', 'saling', 'sama', 'sama-sama', 'sambil', 'sampai', 'sampai-sampai', 'sampaikan', 'sana', 'sangat', 'sangatlah', 'satu', 'saya', 'sayalah', 'se', 'sebab', 'sebabnya', 'sebagai', 'sebagaimana', 'sebagainya', 'sebagian', 'sebaik', 'sebaik-baiknya', 'sebaiknya', 'sebaliknya', 'sebanyak', 'sebegini', 'sebegitu', 'sebelum', 'sebelumnya', 'sebenarnya', 'seberapa', 'sebesar', 'sebetulnya', 'sebisanya', 'sebuah', 'sebut', 'sebutlah', 'sebutnya', 'x', 'secara', 'secukupnya', 'sedang', 'sedangkan', 'sedemikian', 'sedikit', 'sedikitnya', 'seenaknya', 'segala', 'segalanya', 'segera', 'seharusnya', 'sehingga', 'seingat', 'sejak', 'sejauh', 'sejenak', 'sejumlah', 'sekadar', 'sekadarnya', 'sekali', 'sekali-kali', 'sekalian', 'sekaligus', 'sekalipun', 'sekarang', 'sekarang', 'sekecil', 'seketika', 'sekiranya', 'sekitar', 'sekitarnya', 'sekurang-kurangnya', 'sekurangnya', 'sela', 'selain', 'selaku', 'selalu', 'selama', 'selama-lamanya', 'selamanya', 'selanjutnya', 'seluruh', 'seluruhnya', 'semacam', 'semakin', 'semampu', 'semampunya', 'semasa', 'semasih', 'semata', 'semata-mata', 'semaunya', 'sementara', 'semisal', 'semisalnya', 'sempat', 'semua', 'semuanya', 'semula', 'sendiri', 'sendirian', 'sendirinya', 'seolah', 'seolah-olah', 'seorang', 'sepanjang', 'sepantasnya', 'sepantasnyalah', 'seperlunya', 'seperti', 'sepertinya', 'sepihak', 'sering', 'seringnya', 'serta', 'serupa', 'sesaat', 'sesama', 'sesampai', 'sesegera', 'sesekali', 'seseorang', 'sesuatu', 'sesuatunya', 'sesudah', 'sesudahnya', 'setelah', 'indihomo', 'setempat', 'setengah', 'seterusnya', 'setiap', 'setiba', 'setibanya', 'setidak-tidaknya', 'setidaknya', 'setinggi', 'seusai', 'sewaktu', 'siap', 'siapa', 'siapakah', 'siapapun', 'sini', 'sinilah', 'soal', 'soalnya', 'suatu', 'sudah', 'sudahkah', 'sudahlah', 'supaya', 'tadi', 'tadinya', 'tahu', 'tahun', 'tak', 'tambah', 'tambahnya', 'tampak', 'tampaknya', 'tandas', 'tandasnya', 'tanpa', 'tanya', 'tanyakan', 'tanyanya', 'tapi', 'tegas', 'tegasnya', 'telah', 'tempat', 'tengah', 'tentang', 'tentu', 'tentulah', 'tentunya', 'tepat', 'terakhir', 'terasa', 'terbanyak', 'terdahulu', 'terdapat', 'terdiri', 'terhadap', 'terhadapnya', 'teringat', 'teringat-ingat', 'terjadi', 'terjadilah', 'terjadinya', 'terkira', 'terlalu', 'terlebih', 'terlihat', 'termasuk', 'ternyata', 'tersampaikan', 'tersebut', 'tersebutlah', 'tertentu', 'tertuju', 'terus', 'terutama', 'tetap', 'tetapi', 'tiap', 'wifinya', 'ngeles', 'paraahhhh', 'notifnya', 'tut','tiba', 'tiba-tiba', 'tidak', 'tidakkah', 'tidaklah', 'tiga', 'tinggi', 'toh', 'tunjuk', 'turut', 'tutur', 'tuturnya', 'ucap', 'ucapnya', 'ujar', 'ujarnya', 'umum', 'umumnya', 'ungkap', 'ungkapnya', 'untuk', 'usah', 'usai', 'waduh', 'wah', 'wahai', 'waktu', 'waktunya', 'walau', 'walaupun', 'wong', 'yaitu', 'yakin', 'yakni', 'yang', 'telahelebihi', 'mb', 'ditindaklanjuti', 'merrsahkan', 'sdh', 'cuman', 'renew', 'fingerprint'
}

stop_words.update(indihome_stopwords)

def remove_duplicate_words(sentence):
    words = sentence.split()
    unique_words = list(dict.fromkeys(words))
    return ' '.join(unique_words)

def filtering(text):
    filtered_text = [word for word in text if word not in stop_words]
    return filtered_text

data_clean['token_filter'] = data_clean['token'].apply(filtering, remove_duplicate_words)
data_clean.head(10)

### **Stemming**

In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}
hitung=0

for document in data_clean['token_filter']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    hitung+=1

def get_stemmed_term(document):
    return [term_dict[term] for term in document]

data_clean['text_stem'] = data_clean['token_filter'].apply(get_stemmed_term)
data_clean.head(10)

In [ ]:
data_clean.to_csv('C:\dedek\data_myindihome\hasil_preprocessing_myindihome.csv', index= False)

### **Ekstraksi Fitur**

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

data_clean = pd.read_csv('C:\dedek\data_myindihome\hasil_preprocessing_myindihome.csv')

# Menginisialisasi TfidfVectorizer
vectorizer = TfidfVectorizer()

# Melakukan fit dan transformasi pada kolom 'text_stem'
text_tfidf = vectorizer.fit_transform(data_clean['text_stem'])

# Mendapatkan nama fitur dari vectorizer
feature_names = np.array(vectorizer.get_feature_names_out())

def get_top_tfidf_features(text_tfidf, feature_names, top_n=10):
    # Menghitung nilai rata-rata TF-IDF untuk setiap term
    avg_tfidf = text_tfidf.mean(axis=0).A1  # .A1 untuk mengubah matriks menjadi array

    # Mengurutkan nilai rata-rata TF-IDF secara menurun
    sorted_tfidf_indices = avg_tfidf.argsort()[::-1]

    # Memilih top_n term dengan nilai TF-IDF tertinggi
    top_features = feature_names[sorted_tfidf_indices][:top_n]
    top_features_tfidf = avg_tfidf[sorted_tfidf_indices][:top_n]

    return pd.DataFrame({'Term': top_features, 'TF-IDF': top_features_tfidf})

# Menampilkan term rank (10 teratas)
top_tfidf_features = get_top_tfidf_features(text_tfidf, feature_names)
print("TF-IDF Features:")
print(top_tfidf_features)

In [ ]:
import joblib
joblib.dump(vectorizer, 'vectorizer_myindihome.pkl')



### **Grid Search**

In [ ]:
import numpy as np
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.tree import DecisionTreeClassifier, export_text, plot_tree

X = text_tfidf
y = data_clean['Label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 10, 20],
    'min_samples_leaf': [1, 10, 20]
}

clf = DecisionTreeClassifier()
grid_search = GridSearchCV(clf, param_grid, verbose=2)

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_

best_model = grid_search.best_estimator_

y_pred = best_model.predict(X_test)

In [ ]:
print(f'Jumlah data training: {X_train.shape[0]}')
print(f'Jumlah data testing: {X_test.shape[0]}')

In [ ]:
import joblib
joblib.dump(best_model, 'model_myindihome.pkl')

### **Evaluasi**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

accuracy = accuracy_score(y_test, y_pred)
confusion_matrix = confusion_matrix(y_test, y_pred)
classification_report = classification_report(y_test, y_pred)

print(grid_search.best_params_)
print(grid_search.best_estimator_)
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report)
sns.heatmap(confusion_matrix, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

plt.figure(figsize=(20,10))
plot_tree(best_model, filled=True, feature_names=X.columns if hasattr(X, 'columns') else None, class_names=True)
plt.title("Decision Tree")
plt.show()

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

feature_names = vectorizer.get_feature_names_out()

word_scores = dict(zip(feature_names, best_model.feature_importances_))

wc = WordCloud(background_color="white", max_words=100, max_font_size=50)
wc.generate_from_frequencies(word_scores)

plt.figure(figsize=(10, 7))
plt.imshow(wc, interpolation="bilinear")
plt.axis("off")
plt.show()